In [1]:
import onnxruntime
import torchvision
import torch.utils.data as data
import torchvision.transforms as transforms

import numpy as np
import time

In [2]:
IMAGENET_DIR = '/home/data/ImageNet'
BATCH_SIZE = 128
LOADER_WORKERS = 4

In [3]:
ort_sess = onnxruntime.InferenceSession('../Flask/Models/resnet34-onnx.onnx',
                                       providers=['CUDAExecutionProvider'])

In [4]:
_input = ort_sess.get_inputs()[0].name
_output = ort_sess.get_outputs()[0].name
_input, _output

('input_0', 'output_0')

In [5]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
dataset = torchvision.datasets.ImageNet(root=IMAGENET_DIR, transform=transform, split='val')
loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=LOADER_WORKERS)

In [7]:
n_top1 = 0
n_top5 = 0
cnt = 0

s = time.time()
pred_tms = []
for images, labels in loader:

    ss = time.time()
    output = ort_sess.run([_output], {_input: images.numpy()})
    output = output[0]
    pred_tms.append(time.time()-ss)

    cnt += output.shape[0]

    top1_id = output.argmax(1)
    top5_id = output.argsort(1)[:, ::-1][:, :5]

    n_top1 += np.equal(top1_id, labels).sum()
    n_top5 += np.max(np.isin(top5_id, labels), 1).sum()

    print(f"\rstep: {cnt}/{len(dataset)}", end='')
    
total_tm = time.time() - s
print()
print(f"top-1:  {n_top1/cnt:0.4f}")
print(f"top-5:  {n_top5/cnt:0.4f}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Total Time: {total_tm:0.4f} ({total_tm/len(dataset):0.4f})")
print(f"Average Prediction Time: {np.mean(pred_tms):0.4f}")

step: 50000/50000
top-1:  0.7330
top-5:  0.9219
Batch Size: 128
Total Time: 47.8250 (0.0010)
Average Prediction Time: 0.0493
